# Running RAPIDS hyperparameter experiments at scale on Amazon SageMaker

##### Import packages and create Amazon SageMaker and Boto3 sessions

In [14]:
import os
import numpy as np
import time
import numpy as np
import sagemaker
import time
import boto3
from sagemaker.s3 import S3Uploader

sess = boto3.Session()
s3 = boto3.resource('s3')
sagemaker_session = sagemaker.Session()
role = 'admin-role' #sagemaker.get_execution_role()

##### Download the higgs-boson dataset

In [ ]:
!mkdir dataset
!wget -P dataset https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz
!gunzip dataset/HIGGS.csv.gz

##### Download the RAPIDS container from Docker hub

In [ ]:
!docker pull rapidsai/rapidsai:cuda10.0-runtime-ubuntu16.04

##### Define hyperparameters: start with best guess values
Find the full list of Random Forest hyperparameters here in the RAPIDS doc page:
<br>
https://rapidsai.github.io/projects/cuml/en/0.11.0/api.html#random-forest

In [15]:
hyperparams={ 
    'n_estimators'       : 15,
    'max_depth'          : 5,
    'n_bins'             : 8,
    'split_criterion'    : 0,      # GINI:0, ENTROPY:1
    'split_algo'         : 0,      # HIST:0 GLOBAL_QUANTILE:1
    'bootstrap'          : 0,      # true: sample with replacement, false: sample without replacement
    'bootstrap_features' : 0,      # true: sample with replacement, false: sample without replacement
    'max_leaves'         : -1,     # unlimited leaves
    'max_features'       : 0.2, 
}

##### Before we run a large scale experiment, test training locally using the SageMaker SDK

##### Extend RAPIDS container by copying the training script and installing SageMaker containers which makes RAPIDS compatible with SageMaker

In [ ]:
!cat docker/Dockerfile

In [22]:
!docker build -t sagemaker-rapids:latest docker

Sending build context to Docker daemon  8.704kB
Step 1/5 : FROM rapidsai/rapidsai:cuda10.0-runtime-ubuntu16.04
 ---> 1ed8b88fe8ba
Step 2/5 : RUN apt-get update && apt-get install -y --no-install-recommends build-essential
 ---> Using cache
 ---> 6a1441d22aeb
Step 3/5 : RUN source activate rapids && pip install sagemaker-containers
 ---> Using cache
 ---> 296deda06849
Step 4/5 : COPY rapids-higgs.py /opt/ml/code/rapids-higgs.py
 ---> 369e0eb38125
Step 5/5 : ENV SAGEMAKER_PROGRAM rapids-higgs.py
 ---> Running in fa149ecdaa38
Removing intermediate container fa149ecdaa38
 ---> e1abadefd521
Successfully built e1abadefd521
Successfully tagged sagemaker-rapids:latest


In [23]:
from sagemaker.estimator import Estimator

train_instance_type = 'local_gpu'
local_data_dir = 'file://./dataset'

rapids_estimator = Estimator(image_name='sagemaker-rapids:latest',
                          role=role,
                          train_instance_count=1,
                          train_instance_type=train_instance_type,
                          hyperparameters=hyperparams,
                          metric_definitions=[{'Name': 'test_acc', 'Regex': 'test_acc: ([0-9\\.]+)'}])

In [24]:
%%time
rapids_estimator.fit({'dataset': local_data_dir})

Creating tmptgfvyt34_algo-1-tx2g3_1 ... 
Attaching to tmptgfvyt34_algo-1-tx2g3_12mdone
algo-1-tx2g3_1  | 2020-05-14 23:41:25,921 sagemaker-containers INFO     Invoking user script
algo-1-tx2g3_1  | 
algo-1-tx2g3_1  | Training Env:
algo-1-tx2g3_1  | 
algo-1-tx2g3_1  | {
algo-1-tx2g3_1  |     "additional_framework_parameters": {},
algo-1-tx2g3_1  |     "channel_input_dirs": {
algo-1-tx2g3_1  |         "dataset": "/opt/ml/input/data/dataset"
algo-1-tx2g3_1  |     },
algo-1-tx2g3_1  |     "current_host": "algo-1-tx2g3",
algo-1-tx2g3_1  |     "framework_module": null,
algo-1-tx2g3_1  |     "hosts": [
algo-1-tx2g3_1  |         "algo-1-tx2g3"
algo-1-tx2g3_1  |     ],
algo-1-tx2g3_1  |     "hyperparameters": {
algo-1-tx2g3_1  |         "n_estimators": 15,
algo-1-tx2g3_1  |         "max_depth": 5,
algo-1-tx2g3_1  |         "n_bins": 8,
algo-1-tx2g3_1  |         "split_criterion": 0,
algo-1-tx2g3_1  |         "split_algo": 0,
algo-1-tx2g3_1  |         "bootstrap": 0,
algo-1-tx2g3_1  |         "b

In [ ]:
region = boto3.Session().region_name
account = boto3.client('sts').get_caller_identity().get('Account')
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-rapids:latest'.format(account, region)

In [ ]:
!aws ecr create-repository --repository-name sagemaker-rapids
!$(aws ecr get-login --no-include-email --region {region})
!docker tag sagemaker-rapids:latest {image}
!docker push {image}

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'n_estimators'        : IntegerParameter(10, 200), 
    'max_depth'           : IntegerParameter(1, 22),
    'n_bins'              : IntegerParameter(5, 24),
    'split_criterion'     : CategoricalParameter([0, 1]),
    'split_algo'          : CategoricalParameter([0, 1]),
    'bootstrap'           : CategoricalParameter([True, False]),
    'bootstrap_features'  : CategoricalParameter([True, False]),
    'max_features'        : ContinuousParameter(0.01, 0.5),
}

In [ ]:
from sagemaker.estimator import Estimator

train_instance_type = 'ml.p3.2xlarge'
rapids_estimator = Estimator(image_name=image,
                          role=role,
                          train_instance_count=1,
                          train_instance_type=train_instance_type,
                          hyperparameters=hyperparams,
                          metric_definitions=[{'Name': 'test_acc', 'Regex': 'test_acc: ([0-9\\.]+)'}])

In [ ]:
tuner = HyperparameterTuner(rapids_estimator,
                            objective_metric_name='test_acc',
                            hyperparameter_ranges=hyperparameter_ranges,
                            strategy='Bayesian',
                            max_jobs=1,
                            max_parallel_jobs=1,
                            objective_type='Maximize',
                            metric_definitions=[{'Name': 'test_acc', 'Regex': 'test_acc: ([0-9\\.]+)'}])

##### Upload it to the default SageMaker bucket on Amazon S3

In [ ]:
s3_data_dir = sagemaker_session.upload_data(path='dataset', key_prefix='dataset/higgs-dataset')

In [ ]:
job_name = 'rapidsHPO' + time.strftime('%Y-%m-%d-%H-%M-%S-%j', time.gmtime())
tuner.fit({'dataset': s3_data_dir}, job_name=job_name)

## Clean up

- Delete S3 buckets and files you don't need
- Kill training jobs that you don't want running
- Delete container images and the repository you just created

In [ ]:
aws ecr delete-repository --force --repository-name